# Topic Modeling using NMF and KLBNMF on Twitter Accounts

In this notebook, we applied a probabilistic matrix decomposition algorithm called NMF and its variation KLBNMF to detect what kind of topics did a specific user tweet about and what are those topics about.


## Natural Language Processing (NLP)

The language of twitter is generally close to daily language. People share their ideas and emotions at any time of the day. Other than normal texts, tweets can include hashtags, emoticons, pictures, videos, gifs, urls etc. Even normal text part of the tweets may consist of misspelled words. Apart from these, one user may tweet in lots of language. For example, one tweet may be in Turkish, and another one in English. So we need to make a cleanup before using those tweets. The list of applied processes:

* Remove Twitter Accounts that has less than 2000 words in their tweets
* Remove URLs
* Tokenization
* Stop words
* Remove non-English accounts
* Delete accounts whose number of left tokens are less than 200
* Stemming
* Remove words that appears at most 10 timse in the whole corpus

Importing the necessary libraries.

In [1]:
import langid
import logging
import nltk
import numpy as np
import re
import os
import sys
import time
from collections import defaultdict
from string import digits
import pyLDAvis.gensim
import pyLDAvis.sklearn
from gensim import corpora, models, similarities, matutils
import networkx as nx
import string
import math
import pickle

from time import time

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups
from sklearn.cluster import KMeans

from collections import Counter

import scipy.io
from scipy import sparse

C:\Users\Burki\Anaconda3\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\Burki\Anaconda3\lib\site-packages\numpy\lib\utils.py:99: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  warnings.warn(depdoc, DeprecationWarning)


In [2]:
def totalWordCount(tList):
    totalWords = 0
    for tt in tList:
        totalWords += len(tt)
    return totalWords

def totalWordCount2(corpus):
    totalWords = 0
    for corp in corpus:
        for c in corp:
            totalWords += c[1]
    return totalWords

#### Read and Remove Twitter Accounts that has less than 2000 words in their tweets

We have already collected tweets of random 900 followers of TRTWorld's twitter account. You can also find those Twitter API codes in this repo.

Here we are reading each user's tweets from files and saving them into a list (tweetList) if the number of words in the file greater than 2000 words.

In [5]:
tweetsList = []
userList = []

for file in os.listdir("tweets3"):
    path = "tweets3\\" + file
    f = open(path, 'r', encoding='utf-8')
    fread = f.read()
    if (len(fread.split()) > 2000):
        tweetsList.append(fread)
        userList.append(file[0:len(file)-4])
    f.close()

print("Number of Users: %d" %(len(tweetsList)))
print("Total Number of Words: %d" %(totalWordCount(tweetsList)))

Number of Users: 1117
Total Number of Words: 328154028


#### Remove URLs

We have removed all urls which are starting with "http://" or "https://. So we excluded all pictures, videos, gifs etc. from the text.

In [6]:
def remove_urls(text):
    text = re.sub(r"(?:\@|http?\://)\S+", "", text)
    text = re.sub(r"(?:\@|https?\://)\S+", "", text)
    return text

def doc_rm_urls():
    return [ remove_urls(tweets) for tweets in tweetsList]

tweetsList = doc_rm_urls()

print("Total Number of Words: %d" %(totalWordCount(tweetsList)))

Total Number of Words: 236316529


#### Tokenization
Tokenization is basically process of splitting text into words, phrases or other meaningful elements called tokens. We words as our tokens. To better process the text and to create a dictionary and a corpus we tokenized and converted to lower case all the tweets. We used nltk library with regexp to tokenize. 

In [7]:
# This returns a list of tokens / single words for each user
def tokenize_tweet():
    '''
        Tokenizes the raw text of each document
    '''
    tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
    return [ tokenizer.tokenize(t.lower()) for t in tweetsList]

tweetsList = tokenize_tweet()

print("Total Number of Words: " + str(totalWordCount(tweetsList)))

Total Number of Words: 38119510


#### Stop words
Stop words usually refer to the most common words in a language. So being common makes stopwords less effective and sometimes misleading while making decisions. Thus generally stop words are words which are filtered out. We used nltk library to obtain general English stop words, also we determined some words ourselves and also added one and two character words from tweets to stop words.

In [8]:
# Remove stop words
stoplist_tw=['amp','get','got','hey','hmm','hoo','hop','iep','let','ooo','par',
            'pdt','pln','pst','wha','yep','yer','aest','didn','nzdt','via',
            'one','com','new','like','great','make','top','awesome','best',
            'good','wow','yes','say','yay','would','thanks','thank','going',
            'new','use','should','could','best','really','see','want','nice',
            'while','know', 'rt', 'http', 'https']

stoplist  = set(nltk.corpus.stopwords.words("english") + stoplist_tw)

## bu sayi olayini yapicaksak 3d olayina dikkat et
tweetsList = [[token for token in tweets if token not in stoplist and len(token) > 1]
                for tweets in tweetsList]

print("Total Number of Words: " + str(totalWordCount(tweetsList)))

Total Number of Words: 22240226


#### Remove non-English accounts
It is an extension process to removing non-English words. After removing non-English words from tweets, we removed accounts from our corpus whose tweets are majorly not in English. We used a library called langid to detect English accounts.

In [9]:
# Delete Accounts whose tweets are not majorly in English
tweetsList2 = [tweets for tweets in tweetsList if langid.classify(' '.join(tweets))[0] == 'en']

print("Number of Users: " + str(len(tweetsList2)))
print("Total Number of Words: " + str(totalWordCount(tweetsList2)))

Number of Users: 998
Total Number of Words: 19530500


#### Delete accounts whose number of left tokens are less than 200
After all those preprocessing on tweets, we have removed lots of words from original tweets. Some of the accounts, which are possibly not majorly in English but still includes English words, effected more but still existed in the corpus. So to eliminate those misleading accounts from the corpus we deleted accounts whose number of left tokens are less than 200.

In [10]:
# Delete Accounts whose length of tokenized tweets are less than 200
tweetsList2 = [tweets for tweets in tweetsList2 if len(tweets) > 200]
print("Number of Users: " + str(len(tweetsList2)))

Number of Users: 998


#### Remap the User IDs to tweets
In the last two steps we have deleted some accounts. So we need to remap the user ids to the tweets. You can reach tweets from tweetsList2 and users from userList2.

In [11]:
userList2 = []
for i in range(len(tweetsList2)):
    for j in range(i, len(tweetsList)):
        if tweetsList2[i] == tweetsList[j]:
            userList2.append(userList[j])
            break

#### Stemming
For grammatical reasons, documents are going to use different forms of a word, such as organize, organizes, and organizing. Additionally, there are families of derivationally related words with similar meanings, such as democracy, democratic, and democratization. The goal of stemming is to reduce inflectional forms and sometimes derivationally related forms of a word to a common base form. nltk library has mainly 3 kinds of stemming tools for English: lancaster, porter and snowball. We chose Snowball stemmer because it uses a more developed algorithm then Porter Stemmer (Snowball is also called as Porter2) and less aggressive than Lancaster.

In [12]:
# Porter Stemmer and Snowball Stemmer (Porter2) - We useed Snowball Stemmer
# http://stackoverflow.com/questions/10554052/what-are-the-major-differences-and-benefits-of-porter-and-lancaster-stemming-alg

sno = nltk.stem.SnowballStemmer('english')

tweetsList2 = [[sno.stem(token) for token in tweets]
          for tweets in tweetsList2]

print("Total Number of Words: " + str(totalWordCount(tweetsList2)))

Total Number of Words: 19530500


### Dictionary and Corpus

In [13]:
# Build a dictionary where for each document each word has its own id
dictionary = corpora.Dictionary(tweetsList2)
dictionary.compactify()

# Build the corpus: vectors with occurence of each word for each document
# convert tokenized documents to vectors
corpus = [dictionary.doc2bow(tweets) for tweets in tweetsList2]

print(dictionary)

Dictionary(375149 unique tokens: ['fcpsnew', 'projectaspir', 'malagamak', 'gwendolyn', 'amputeefit']...)


In [16]:
# Removing words that appears at most 10 times in the whole corpus

dictCtr = np.zeros(len(dictionary))

for c in corpus:
    for tuples in c:
        dictCtr[tuples[0]] = dictCtr[tuples[0]] + tuples[1]
        
badids = []
for i in range(len(dictCtr)):
    if dictCtr[i] < 11:
        badids.append(i)
        
        
dictionary.filter_tokens(bad_ids=badids)
dictionary.compactify()

corpus = [dictionary.doc2bow(tweets) for tweets in tweetsList2]

print(dictionary)

Dictionary(45262 unique tokens: ['generalelect', 'pva', 'busker', '60fps', 'anxieti']...)


In [17]:
tweetList = []

for c in corpus:
    str = ''
    for tokens in c:
        str = str + ((dictionary[tokens[0]]+' ') * tokens[1])
    tweetList.append(str)

print("Number of Users: %d"  %(len(tweetList)))
print("Total Number of Words: %d" %(totalWordCount2(corpus)))

Number of Users: 998
Total Number of Words: 18874173


# Word2Vec

In [18]:
wordModel = models.Word2Vec(tweetsList2, size=30, window=5, min_count=11, workers=4)

print(wordModel)

Word2Vec(vocab=45262, size=30, alpha=0.025)


In [19]:
#print(len(wordModel.wv.index2word))
vocab = wordModel.wv.index2word
wordvectors = wordModel.wv[vocab]

In [20]:
kmeansList = np.asarray(wordvectors).astype(np.float64)

kmeans = KMeans(n_clusters=2000).fit(kmeansList)

In [21]:
clusters = {}
labels = {}
centers = []
inVocab = {}

for i in range(0,2000):
    clusters[i] = []

for i, label in enumerate(kmeans.labels_):
    clusters[label].append(vocab[i])
    labels[vocab[i]] = label
    
for c in kmeans.cluster_centers_:
    centers.append(wordModel.similar_by_vector(c)[0][0])
    
for v in vocab:
    inVocab[v] = 1

In [22]:
# Change words in tweets with their cluster center words
tweets2 = [[centers[labels[r]] for r in row if r in inVocab]
          for row in tweetsList2]

### Dictionary and Corpus

To properly use the Twitter data that we have preprocessed, we need to put into a shape that will be understandable by Topic Modeling algorithms. Bag-of-words representation is perfect fit for those kind of algorithms. In bag-of-words we first created a dictionary which consists of all the words from our preprocessed twitter data as values and their ids as keys. Then we created our corpus. Each element of the corpus corresponds to one Twitter account. Each element consists tuples which includes dictionary id of words and the number of that words' occurrences in that account. We used a very useful python library called Gensim to create our dictionary and corpus.

In [23]:
# Build a dictionary where for each document each word has its own id
dictionaryVW = corpora.Dictionary(tweets2)
dictionaryVW.compactify()

# Build the corpus: vectors with occurence of each word for each document
# convert tokenized documents to vectors
corpusVW = [dictionaryVW.doc2bow(tweets) for tweets in tweets2]

print(dictionaryVW)

Dictionary(1991 unique tokens: ['sale', 'star', 'tutori', 'note', 'construct']...)


In [24]:
# Normalize word counts by dividing it to the number of elements in its cluster
corpusVW = [[(r[0], int(math.ceil(r[1]/ len(clusters[labels[dictionaryVW[r[0]]]]))) ) for r in row]
          for row in corpusVW]

In [25]:
tweetListVW = []

for c in corpusVW:
    str = ''
    for tokens in c:
        str = str + ((dictionaryVW[tokens[0]]+' ') * tokens[1])
    tweetListVW.append(str)

print("Number of Users: %d"  %(len(tweetListVW)))
print("Total Number of Words: %d" %(totalWordCount2(corpusVW)))

Number of Users: 998
Total Number of Words: 10401128


In [50]:
#clusters

In [44]:
dictionary.save('database/normal.dict')
corpora.MmCorpus.serialize('database/normal.mm', corpus)
with open('database/normal.pickle', 'wb') as f:
    pickle.dump([tweetList, userList2], f)

In [45]:
dictionaryVW.save('database/w2v.dict')
corpora.MmCorpus.serialize('database/w2v.mm', corpusVW)
with open('database/w2v.pickle', 'wb') as f:
    pickle.dump([tweetListVW, userList2], f)

In [3]:
dictionary = corpora.Dictionary.load('database/normal.dict')
corpus = corpora.MmCorpus('database/normal.mm')
with open('database/normal.pickle', 'rb') as f:
    tweetList, userList2 = pickle.load(f)

In [4]:
dictionaryVW = corpora.Dictionary.load('database/w2v.dict')
corpusVW = corpora.MmCorpus('database/w2v.mm')
with open('database/w2v.pickle', 'rb') as f:
    tweetListVW, userList2 = pickle.load(f)

# KLBNMF Implementation

In [5]:
# %load gnmf_solvebynewton.py
from __future__ import division
import numpy as np
import scipy as sp
from scipy import special
import numpy.matlib as M

def gnmf_solvebynewton(c, a0 = None):

    if a0 is None:
        a0 = 0.1 * np.ones(np.shape(c))

    M, N = np.shape(a0)
    if len(np.shape(c)) == 0:
        Mc , Nc = 1,1
    else:
        Mc, Nc = np.shape(c)



    a = None
    cond = 0

    if (M == Mc and N == Nc):
        a = a0
        cond = 1

    elif (Mc == 1 and Nc >1):
        cond = 2
        a = a0[0,:]
    elif (Mc > 1 and Nc == 1):
        cond = 3
        a = a0[:,0]
    elif (Mc == 1 and Nc == 1):
        cond = 4
        a = a0[0,0]

    a2 = None
    for index in range(10):
        a2 = a - (np.log(a) - special.polygamma(0,a) + 1 - c) / (1/a - special.polygamma(1,a))
        idx = np.where(a2<0)
        if len(idx[0]) > 0:
            if isinstance(a, float):
                a2 = a / 2
            else:
                a2[idx] = a[idx] / 2
        a = a2

    if(cond == 2):
        a = M.repmat(a,M,1)
    elif(cond == 3):
        a = M.repmat(a,1,N)
    elif(cond == 4):
        a = a * np.ones([M,N])

    return a


In [6]:
# %load gnmf_vb_poisson_mult_fast.py
from __future__ import division
import numpy as np
import scipy as sp
import math
from scipy import special
import numpy.matlib as M

def gnmf_vb_poisson_mult_fast(x,
                            a_tm,
                            b_tm,
                            a_ve,
                            b_ve,
                            EPOCH =1000,
                            Method = 'vb',
                            Update = np.inf,
                            tie_a_ve = 'clamp',
                            tie_b_ve = 'clamp',
                            tie_a_tm = 'clamp',
                            tie_b_tm = 'clamp',
                            print_period = 500
                            ):

    # Result initialiation
    g = dict()
    g['E_T'] = None
    g['E_logT'] = None
    g['E_V'] = None
    g['E_logV'] = None
    g['Bound'] = None
    g['a_ve'] = None
    g['b_ve'] = None
    g['a_tm'] = None
    g['b_tm'] = None

    logm = np.vectorize(math.log)
    W = x.shape[0]
    K = x.shape[1]
    I = b_tm.shape[1]

    M = ~np.isnan(x)
    X = np.zeros(x.shape)
    X[M] = x[M]

    t_init = np.random.gamma(a_tm, b_tm/a_tm)
    v_init = np.random.gamma(a_ve, b_ve/a_ve)
    L_t = t_init
    L_v = v_init
    E_t = t_init
    E_v = v_init
    Sig_t = t_init
    Sig_v = v_init

    B = np.zeros([1,EPOCH])
    gammalnX = special.gammaln(X+1)

    for e in range(1,EPOCH+1):

        LtLv = L_t.dot(L_v)
        tmp = X / (LtLv)
        #check Tranpose
        Sig_t = L_t * (tmp.dot(L_v.T))
        Sig_v = L_v * (L_t.T.dot(tmp))

        alpha_tm = a_tm + Sig_t
        beta_tm = 1/((a_tm/b_tm) + M.dot(E_v.T))
        E_t = alpha_tm * (beta_tm)

        alpha_ve = a_ve + Sig_v
        beta_ve = 1/((a_ve/b_ve) + E_t.T.dot(M))

        E_v = alpha_ve * (beta_ve)
        # Compute the bound
        if(e%10 == 1):
            print("*", end='')
        if(e%print_period == 1 or e == EPOCH):
            g['E_T'] = E_t
            g['E_logT'] = logm(L_t)
            g['E_V'] = E_v
            g['E_logV'] = logm(L_v)

            g['Bound'] = -np.sum(np.sum(M * (g['E_T'].dot(g['E_V'])) + gammalnX))\
                        + np.sum(np.sum(-X * ( ((L_t * g['E_logT']).dot(L_v) + L_t.dot(L_v * g['E_logV']))/(LtLv) - logm(LtLv) ) ))\
                        + np.sum(np.sum((-a_tm/b_tm)* g['E_T'] - special.gammaln(a_tm) + a_tm * logm(a_tm /b_tm)))\
                        + np.sum(np.sum((-a_ve/b_ve)* g['E_V'] - special.gammaln(a_ve) + a_ve * logm(a_ve /b_ve)))\
                        + np.sum(np.sum( special.gammaln(alpha_tm) + alpha_tm * logm(beta_tm) + 1))\
                        + np.sum(np.sum(special.gammaln(alpha_ve) + alpha_ve * logm(beta_ve) + 1 ))

            g['a_ve'] = a_ve
            g['b_ve'] = b_ve
            g['a_tm'] = a_tm
            g['b_tm'] = b_tm

            print()
            print( g['Bound'], a_ve.flatten()[0], b_ve.flatten()[0], a_tm.flatten()[0], b_tm.flatten()[0])
        if (e == EPOCH):
            break;
        L_t = np.exp(special.psi(alpha_tm)) * beta_tm
        L_v = np.exp(special.psi(alpha_ve)) * beta_ve

        Z = None
        if( e> Update):
            if(not tie_a_tm == 'clamp' ):
                Z = (E_t / b_tm) - (logm(L_t) - logm(b_tm))
                if(tie_a_tm == 'clamp'):
                    a_tm = gnmf_solvebynewton(Z,a0=a_tm)
                elif(tie_a_tm == 'rows'):
                    a_tm = gnmf_solvebynewton(np.sum(Z,0)/W, a0=a_tm)
                elif(tie_a_tm == 'cols'):
                    a_tm = gnmf_solvebynewton(np.sum(Z,1)/I, a0=a_tm)
                elif(tie_a_tm == 'tie_all'):
                    #print(np.sum(Z)/(W * I))
                    #print(a_tm)
                    a_tm = gnmf_solvebynewton(np.sum(Z)/(W * I), a0=a_tm)

            if(tie_b_tm == 'free'):
                b_tm = E_t
            elif(tie_b_tm == 'rows'):
                b_tm = M.repmat(np.sum(a_tm * E_t,0)/np.sum(a_tm,0),W,1)
            elif(tie_b_tm == 'cols'):
                b_tm = M.repmat(np.sum(a_tm * E_t,1)/np.sum(a_tm,1),1,I)
            elif(tie_b_tm == 'tie_all'):
                b_tm = (np.sum(a_tm*E_t)/ np.sum(a_tm)) * np.ones([W,I])

            if(not tie_a_ve == 'clamp' ):
                Z = (E_v / b_ve) - (logm(L_v) - logm(b_ve))
                if(tie_a_ve == 'clamp'):
                    a_ve = gnmf_solvebynewton(Z,a_ve)
                elif(tie_a_ve == 'rows'):
                    a_ve = gnmf_solvebynewton(np.sum(Z,0)/I, a0=a_ve)
                elif(tie_a_ve == 'cols'):
                    a_ve = gnmf_solvebynewton(np.sum(Z,1)/K, a0=a_ve)
                elif(tie_a_ve == 'tie_all'):
                    a_ve = gnmf_solvebynewton(np.sum(Z)/(I * K), a0=a_ve)

            if(tie_b_ve == 'free'):
                b_ve = E_v
            elif(tie_b_ve == 'rows'):
                b_ve = M.repmat(np.sum(a_ve * E_v,0)/np.sum(a_ve,0),I,1)
            elif(tie_b_tm == 'cols'):
                b_ve = M.repmat(np.sum(a_ve * E_v,1)/np.sum(a_ve,1),1,K)
            elif(tie_b_tm == 'tie_all'):
                b_ve = (np.sum(a_ve*E_v)/ np.sum(a_ve)) * np.ones([I,K])
    return g


# sklearn NMF-KLBNMF

In [7]:
n_topics = 17
n_top_words = 7
n_top_topics = 3

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        sm = sum(topic)
        print("Topic #%d:" % topic_idx)
        for i in topic.argsort()[:-n_top_words - 1:-1]:
            print("(%s, %lf)  " %(feature_names[i], topic[i]/sm), end='')
        print()
    print()
    
def print_top_words2(H, feature_names, n_top_words):
    for topic_idx, topic in enumerate(H):
        sm = sum(topic)
        print("Topic #%d:" % topic_idx)
        for i in topic.argsort()[:-n_top_words - 1:-1]:
            print("(%s, %lf)  " %(feature_names[i], topic[i]/sm), end='')
        print()
    print()
    
def print_top_topics(doc_topic, user, n_top_topics):
    for i in doc_topic[user].argsort()[:-n_top_topics - 1:-1]:
        print("(%d, %lf)  " %(i, doc_topic[user][i]), end='')
    
def topicAndWords(model, doc_topic, user, feature_names):
    model_comp = model.components_
    for i in doc_topic[user].argsort()[:-3 - 1:-1]:
        print("(%d, %lf)  " %(i, doc_topic[user][i]), end='')
        sm = sum(model_comp[i])
        for j in model_comp[i].argsort()[:-3 - 1:-1]:
            print("(%s, %lf)  " %(feature_names[j], model_comp[i][j]/sm), end='')
        print()

## Normal NMF

In [8]:
n_samples = len(tweetList)
n_features = len(dictionary)

# Use tf-idf features for NMF.
print("Extracting tf-idf features for NMF...")

tfidf_vectorizer = TfidfVectorizer(max_features=n_features)
t0 = time()
tfidf = tfidf_vectorizer.fit_transform(tweetList)
print("done in %0.3fs." % (time() - t0))

# Fit the NMF model
print("Fitting the NMF model with tf-idf features, " "n_samples=%d and n_features=%d..." % (n_samples, n_features))
t0 = time()
nmf = NMF(n_components=n_topics, random_state=1, alpha=.1, l1_ratio=.5).fit(tfidf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in NMF model:")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)

#http://nbviewer.jupyter.org/github/bmabey/pyLDAvis/blob/master/notebooks/sklearn.ipynb#topic=0&lambda=1&term=
nmf_vis_data = pyLDAvis.sklearn.prepare(nmf, tfidf, tfidf_vectorizer)
pyLDAvis.display(nmf_vis_data)

Extracting tf-idf features for NMF...
done in 11.119s.
Fitting the NMF model with tf-idf features, n_samples=998 and n_features=45262...
done in 9.387s.

Topics in NMF model:
Topic #0:
(work, 0.004914)  (time, 0.004630)  (look, 0.004345)  (think, 0.003899)  (peopl, 0.003432)  (day, 0.003389)  (need, 0.003280)  
Topic #1:
(data, 0.241154)  (scienc, 0.044569)  (analyt, 0.036594)  (learn, 0.036334)  (big, 0.031454)  (scientist, 0.026678)  (machin, 0.022504)  
Topic #2:
(bigdata, 0.154639)  (analyt, 0.068607)  (data, 0.043607)  (iot, 0.042267)  (big, 0.025896)  (busi, 0.017473)  (market, 0.015943)  
Topic #3:
(learn, 0.040745)  (deep, 0.024388)  (neural, 0.022234)  (paper, 0.018215)  (deeplearn, 0.017165)  (machin, 0.015572)  (network, 0.013196)  
Topic #4:
(3dprint, 0.249377)  (3d, 0.125076)  (print, 0.101585)  (printer, 0.035551)  (3dprinter, 0.033668)  (design, 0.022039)  (makerbot, 0.016994)  
Topic #5:
(arduino, 0.336034)  (maker, 0.035116)  (raspberrypi, 0.033419)  (shield, 0.026470)

## Normal NMF with word2vec

In [20]:
n_samples = len(tweetListVW)
n_features = len(dictionaryVW)

# Use tf-idf features for NMF.
print("Extracting tf-idf features for NMF...")

tfidf_vectorizerWV = TfidfVectorizer(max_features=n_features)
t0 = time()
tfidfWV = tfidf_vectorizerWV.fit_transform(tweetListVW)
print("done in %0.3fs." % (time() - t0))

# Fit the NMF model
print("Fitting the NMF model with tf-idf features, " "n_samples=%d and n_features=%d..." % (n_samples, n_features))
t0 = time()
nmfWV = NMF(n_components=n_topics, random_state=1, alpha=.1, l1_ratio=.5).fit(tfidfWV)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in NMF model:")
tfidf_feature_names = tfidf_vectorizerWV.get_feature_names()
print_top_words(nmfWV, tfidf_feature_names, n_top_words)

#http://nbviewer.jupyter.org/github/bmabey/pyLDAvis/blob/master/notebooks/sklearn.ipynb#topic=0&lambda=1&term=
nmf_vis_data = pyLDAvis.sklearn.prepare(nmfWV, tfidfWV, tfidf_vectorizerWV)
pyLDAvis.display(nmf_vis_data)

Extracting tf-idf features for NMF...
done in 6.401s.
Fitting the NMF model with tf-idf features, n_samples=998 and n_features=1991...
done in 2.732s.

Topics in NMF model:
Topic #0:
(day, 0.008128)  (time, 0.007678)  (peopl, 0.006849)  (done, 0.006358)  (thing, 0.006102)  (us, 0.005997)  (use, 0.005837)  
Topic #1:
(data, 0.183956)  (analyt, 0.041112)  (scienc, 0.028836)  (big, 0.025990)  (learn, 0.021398)  (scientist, 0.017654)  (bigdata, 0.015501)  
Topic #2:
(bigdata, 0.232107)  (machinelearn, 0.118212)  (analyt, 0.073407)  (datamin, 0.051580)  (artificialintellig, 0.037845)  (iot, 0.031193)  (big, 0.021858)  
Topic #3:
(learn, 0.055295)  (deep, 0.029912)  (neural, 0.025559)  (machinelearn, 0.024601)  (machin, 0.024289)  (paper, 0.022535)  (artificialintellig, 0.020335)  
Topic #4:
(3dprint, 0.273420)  (3d, 0.138683)  (print, 0.121548)  (printer, 0.044875)  (3dprinter, 0.036580)  (design, 0.030340)  (manufactur, 0.019320)  
Topic #5:
(arduino, 0.399388)  (kit, 0.042451)  (shield, 0

# KLBNMF

In [9]:
tfidfDense = tfidf.todense()
tfidfDense2 = tfidfDense*10000
#idx = np.where(tfidfDense2>100)
#print(len(idx[0]))

W = tfidf.shape[0]
K = tfidf.shape[1]
I = n_topics

a_tm = 1 * np.ones([W,I])
b_tm = np.ones([W,I])
a_ve = np.ones([I,K])
b_ve = 8 * np.ones([I,K])

#T = np.random.gamma(a_tm,b_tm)
#V = np.random.gamma(a_ve,b_ve)

#x = np.random.poisson(T.dot(V))

#idx = np.where(x>100)
#print(len(idx[0]))

t0 = time()

klbnmf = gnmf_vb_poisson_mult_fast(np.asarray(tfidfDense2),a_tm,b_tm,a_ve,b_ve,
                                EPOCH=500,
                                Update =10,
                                tie_a_ve='tie_all',
                                tie_b_ve='tie_all',
                                tie_a_tm='tie_all',
                                tie_b_tm='tie_all')

print("done in %0.3fs." % (time() - t0))

*
-736129414.445 1.0 8.0 1.0 1.0
*************************************************
-640927794.57 0.0520008057635 7.97555284127 0.0665246518457 0.0299543017417
done in 643.631s.


## 10000*NMF

In [10]:
# Fit the NMF model
print("Fitting the NMF model with tf-idfDense2 features, " "n_samples=%d and n_features=%d..." % (n_samples, n_features))
t0 = time()
nmf2 = NMF(n_components=n_topics, random_state=1, alpha=.1, l1_ratio=.5).fit(tfidfDense2)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in NMF model:")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf2, tfidf_feature_names, n_top_words)

tfidfDense2sparse = sparse.csr_matrix(tfidfDense2)

#http://nbviewer.jupyter.org/github/bmabey/pyLDAvis/blob/master/notebooks/sklearn.ipynb#topic=0&lambda=1&term=
nmf_vis_data = pyLDAvis.sklearn.prepare(nmf2, tfidfDense2sparse, tfidf_vectorizer)
pyLDAvis.display(nmf_vis_data)

Fitting the NMF model with tf-idfDense2 features, n_samples=998 and n_features=1991...
done in 20.781s.

Topics in NMF model:
Topic #0:
(think, 0.003520)  (work, 0.003450)  (time, 0.003387)  (peopl, 0.003068)  (look, 0.002885)  (trump, 0.002738)  (day, 0.002401)  
Topic #1:
(data, 0.058583)  (scienc, 0.011856)  (learn, 0.008871)  (big, 0.008026)  (analyt, 0.007883)  (scientist, 0.007120)  (machin, 0.005906)  
Topic #2:
(ai, 0.067064)  (machinelearn, 0.016721)  (iot, 0.012656)  (artificialintellig, 0.010507)  (intellig, 0.009868)  (deeplearn, 0.009584)  (artifici, 0.009461)  
Topic #3:
(learn, 0.013733)  (deep, 0.007951)  (neural, 0.007388)  (paper, 0.007122)  (machin, 0.005131)  (network, 0.004792)  (model, 0.004653)  
Topic #4:
(3dprint, 0.060767)  (3d, 0.031039)  (print, 0.025450)  (printer, 0.009274)  (3dprinter, 0.008725)  (design, 0.005953)  (makerbot, 0.004819)  
Topic #5:
(arduino, 0.070015)  (raspberrypi, 0.008195)  (maker, 0.008139)  (shield, 0.006154)  (kit, 0.005910)  (iot, 

## KLBNMF

In [11]:
tfidfKL = np.dot(klbnmf['E_T'],klbnmf['E_V'])

# Fit the NMF model
print("Fitting the KLBNMF model with tf-idf(klbnmf['E_T']*klbnmf['E_V']) features, " "n_samples=%d and n_features=%d..." % (n_samples, n_features))
t0 = time()
klbnmf2 = NMF(n_components=n_topics, random_state=1, alpha=.1, l1_ratio=.5).fit(tfidfKL)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in NMF model:")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(klbnmf2, tfidf_feature_names, n_top_words)

tfidfKLsparse = sparse.csr_matrix(tfidfKL)

#http://nbviewer.jupyter.org/github/bmabey/pyLDAvis/blob/master/notebooks/sklearn.ipynb#topic=0&lambda=1&term=
nmf_vis_data = pyLDAvis.sklearn.prepare(klbnmf2, tfidfKLsparse, tfidf_vectorizer)
pyLDAvis.display(nmf_vis_data)

Fitting the KLBNMF model with tf-idf(klbnmf['E_T']*klbnmf['E_V']) features, n_samples=998 and n_features=45262...
done in 20.213s.

Topics in NMF model:
Topic #0:
(data, 0.006582)  (think, 0.005456)  (work, 0.005071)  (time, 0.004175)  (peopl, 0.004022)  (look, 0.003577)  (thing, 0.003496)  
Topic #1:
(data, 0.035618)  (bigdata, 0.027927)  (datasci, 0.022010)  (analyt, 0.017453)  (machinelearn, 0.013725)  (ai, 0.010039)  (iot, 0.008771)  
Topic #2:
(arduino, 0.033056)  (robot, 0.014797)  (maker, 0.006043)  (kit, 0.005478)  (iot, 0.005437)  (project, 0.004892)  (raspberrypi, 0.004839)  
Topic #3:
(learn, 0.012937)  (ai, 0.007429)  (deep, 0.006982)  (neural, 0.006111)  (paper, 0.005890)  (deeplearn, 0.005710)  (machin, 0.005583)  
Topic #4:
(3dprint, 0.035776)  (3d, 0.019450)  (print, 0.017560)  (printer, 0.006301)  (design, 0.005387)  (3dprinter, 0.004917)  (makerbot, 0.004126)  
Topic #5:
(robot, 0.013779)  (drone, 0.013642)  (ai, 0.005539)  (tech, 0.005433)  (iot, 0.004645)  (wearabl,

# KLBNMF with word2vec

In [21]:
tfidfDenseWV = tfidfWV.todense()
tfidfDense2WV = tfidfDenseWV*10000
#idx = np.where(tfidfDense2>100)
#print(len(idx[0]))

W = tfidfWV.shape[0]
K = tfidfWV.shape[1]
I = n_topics

a_tm = 1 * np.ones([W,I])
b_tm = np.ones([W,I])
a_ve = np.ones([I,K])
b_ve = 8 * np.ones([I,K])

#T = np.random.gamma(a_tm,b_tm)
#V = np.random.gamma(a_ve,b_ve)

#x = np.random.poisson(T.dot(V))

#idx = np.where(x>100)
#print(len(idx[0]))

klbnmfWV = gnmf_vb_poisson_mult_fast(np.asarray(tfidfDense2WV),a_tm,b_tm,a_ve,b_ve,
                                EPOCH=1000,
                                Update =10,
                                tie_a_ve='tie_all',
                                tie_b_ve='tie_all',
                                tie_a_tm='tie_all',
                                tie_b_tm='tie_all')

*
-348689703.074 1.0 8.0 1.0 1.0
**************************************************
-301662516.133 0.178982062587 7.98024126501 0.0853115805522 0.441200384068
*************************************************
-301660493.234 0.177644143738 7.96804094317 0.0844545488278 0.441191024482


## 10000 * NMF with word2vec

In [14]:
# Fit the NMF model
print("Fitting the NMF model with tf-idfDense2 features, " "n_samples=%d and n_features=%d..." % (n_samples, n_features))
t0 = time()
nmfWV2 = NMF(n_components=n_topics, random_state=1, alpha=.1, l1_ratio=.5).fit(tfidfDense2WV)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in NMF model:")
tfidf_feature_names = tfidf_vectorizerWV.get_feature_names()
print_top_words(nmfWV2, tfidf_feature_names, n_top_words)

tfidfDense2sparseWV = sparse.csr_matrix(tfidfDense2WV)

#http://nbviewer.jupyter.org/github/bmabey/pyLDAvis/blob/master/notebooks/sklearn.ipynb#topic=0&lambda=1&term=
nmf_vis_data = pyLDAvis.sklearn.prepare(nmfWV2, tfidfDense2sparseWV, tfidf_vectorizerWV)
pyLDAvis.display(nmf_vis_data)

Fitting the NMF model with tf-idfDense2 features, n_samples=998 and n_features=1991...
done in 1.118s.

Topics in NMF model:
Topic #0:
(learn, 0.033572)  (deep, 0.019368)  (paper, 0.018584)  (neural, 0.017396)  (machin, 0.013543)  (model, 0.011572)  (use, 0.010020)  
Topic #1:
(bigdata, 0.151051)  (analyt, 0.069137)  (data, 0.037319)  (big, 0.022213)  (iot, 0.019312)  (datamin, 0.013098)  (nosql, 0.011560)  
Topic #2:
(us, 0.016322)  (day, 0.010561)  (help, 0.009081)  (join, 0.008393)  (check, 0.007993)  (design, 0.007607)  (team, 0.006766)  
Topic #3:
(machinelearn, 0.189468)  (artificialintellig, 0.058158)  (deeplearn, 0.049707)  (datamin, 0.046410)  (learn, 0.042983)  (machin, 0.027649)  (bigdata, 0.025174)  
Topic #4:
(3dprint, 0.152152)  (3d, 0.079335)  (print, 0.069414)  (printer, 0.026479)  (3dprinter, 0.021235)  (design, 0.018078)  (maker, 0.011194)  
Topic #5:
(arduino, 0.153455)  (maker, 0.035527)  (kit, 0.024734)  (raspberrypi, 0.020141)  (project, 0.016893)  (pi, 0.015462) 

## KLBNMF with word2vec

In [22]:
tfidfKLWV = np.dot(klbnmfWV['E_T'],klbnmfWV['E_V'])

# Fit the NMF model
print("Fitting the KLBNMF model with tf-idf(klbnmf['E_T']*klbnmf['E_V']) features, " "n_samples=%d and n_features=%d..." % (n_samples, n_features))
t0 = time()
klbnmfWV2 = NMF(n_components=n_topics, random_state=1, alpha=.1, l1_ratio=.5).fit(tfidfKLWV)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in NMF model:")
tfidf_feature_names = tfidf_vectorizerWV.get_feature_names()
print_top_words(klbnmfWV2, tfidf_feature_names, n_top_words)

tfidfKLsparseWV = sparse.csr_matrix(tfidfKLWV)

#http://nbviewer.jupyter.org/github/bmabey/pyLDAvis/blob/master/notebooks/sklearn.ipynb#topic=0&lambda=1&term=
nmf_vis_data = pyLDAvis.sklearn.prepare(klbnmfWV2, tfidfKLsparseWV, tfidf_vectorizerWV)
pyLDAvis.display(nmf_vis_data)

Fitting the KLBNMF model with tf-idf(klbnmf['E_T']*klbnmf['E_V']) features, n_samples=998 and n_features=1991...
done in 0.780s.

Topics in NMF model:
Topic #0:
(time, 0.013646)  (thing, 0.013603)  (peopl, 0.012276)  (day, 0.011863)  (done, 0.011262)  (look, 0.009092)  (take, 0.007558)  
Topic #1:
(bigdata, 0.081065)  (machinelearn, 0.059408)  (data, 0.032551)  (artificialintellig, 0.031902)  (analyt, 0.027554)  (datamin, 0.020429)  (iot, 0.019658)  
Topic #2:
(robot, 0.075723)  (drone, 0.044026)  (uav, 0.011424)  (wearabl, 0.011403)  (tech, 0.010894)  (car, 0.009853)  (omgrobot, 0.009625)  
Topic #3:
(learn, 0.042553)  (deep, 0.025870)  (neural, 0.023508)  (paper, 0.018302)  (machin, 0.015586)  (model, 0.014856)  (use, 0.012216)  
Topic #4:
(3dprint, 0.113673)  (3d, 0.068146)  (print, 0.061105)  (manufactur, 0.026201)  (printer, 0.022611)  (design, 0.019339)  (3dprinter, 0.014940)  
Topic #5:
(data, 0.081904)  (python, 0.057536)  (rstat, 0.034199)  (scienc, 0.023673)  (learn, 0.019013

In [ ]:
print_top_words(nmf, tfidf_vectorizer.get_feature_names(), n_top_words)
print_top_words2(klbnmf['E_V'], tfidf_vectorizer.get_feature_names(), n_top_words)

In [13]:
ids = []
chosenNames = ['andrewyng', 'radbuzzz', 'RoboticsEU', 'karpathy', 'polar3d', 'thearduinoguy']

# @andrewyng => 216939636
ids.append(userList2.index('216939636'))
# @radbuzzz => 28953366 
ids.append(userList2.index('28953366'))
# @RoboticsEU => 335419621
ids.append(userList2.index('335419621'))
# @karpathy => 33836629
ids.append(userList2.index('33836629'))
# @polar3d => 2875670213
ids.append(userList2.index('2875670213'))
# @thearduinoguy => 15392736
ids.append(userList2.index('15392736'))

In [23]:
from operator import itemgetter
#nmfTopics = nmfWV.transform(tfidfWV)
klbnmf2Topics = klbnmf2.transform(tfidfKLsparse)
klbnmfWV2Topics = klbnmfWV2.transform(tfidfKLsparseWV)
#klbnmfWV2, tfidfKLsparseWV, tfidf_vectorizerWV
#nmfWV, tfidfWV, tfidf_vectorizerWV
#ldaTopics = ldaSK.transform(tf)

In [26]:
for i, id in enumerate(ids):
    print(chosenNames[i])
    #print("scikit NMF (v2w)")
    #topicAndWords(nmfWV, nmfTopics, id, tfidf_vectorizerWV.get_feature_names())
    #print()
    print("KLBNMF")
    topicAndWords(klbnmf2, klbnmf2Topics, id, tfidf_vectorizer.get_feature_names())
    print()
    print("KLBNMF (w2v)")
    topicAndWords(klbnmfWV2, klbnmfWV2Topics, id, tfidf_vectorizerWV.get_feature_names())
    print()
    #print("scikit LDA (v2w)")
    #topicAndWords(ldaSK, ldaTopics, id, tf_feature_names)
    #print()
    #print("gensim LDA (v2w)")
    #print(list(reversed(sorted(lda[corpusVW[id]], key=itemgetter(1))))[:n_top_topics])
    #print()

andrewyng
KLBNMF
(3, 12.496066)  (learn, 0.012937)  (ai, 0.007429)  (deep, 0.006982)  
(13, 9.892293)  (manufactur, 0.007536)  (innov, 0.005379)  (robot, 0.005299)  
(6, 6.061536)  (stem, 0.011202)  (edtech, 0.011067)  (code, 0.009961)  

KLBNMF (w2v)
(13, 22.401302)  (data, 0.022629)  (scienc, 0.017825)  (research, 0.016573)  
(12, 6.457801)  (startup, 0.031082)  (busi, 0.022587)  (tech, 0.020072)  
(3, 5.859447)  (learn, 0.042553)  (deep, 0.025870)  (neural, 0.023508)  

radbuzzz
KLBNMF
(13, 14.581962)  (manufactur, 0.007536)  (innov, 0.005379)  (robot, 0.005299)  
(4, 9.780511)  (3dprint, 0.035776)  (3d, 0.019450)  (print, 0.017560)  
(5, 6.421823)  (robot, 0.013779)  (drone, 0.013642)  (ai, 0.005539)  

KLBNMF (w2v)
(4, 16.783202)  (3dprint, 0.113673)  (3d, 0.068146)  (print, 0.061105)  
(16, 12.949837)  (health, 0.011468)  (healthcar, 0.010648)  (innov, 0.009932)  
(1, 4.218421)  (bigdata, 0.081065)  (machinelearn, 0.059408)  (data, 0.032551)  

RoboticsEU
KLBNMF
(14, 18.949887)  

In [ ]:
gensim LDA

andrewyng
[(15, 0.48172061152763984), (10, 0.21645109819341016), (3, 0.092197346963785592)]
'0.044*"learn" + 0.021*"deep" + 0.020*"machin"
'0.013*"trump" + 0.010*"year" + 0.010*"us" 
'0.017*"love" + 0.016*"day" + 0.014*"today"


radbuzzz
[(2, 0.36363761882489798), (10, 0.22657291528259471), (6, 0.15630377228249343)]
'0.087*"3dprint" + 0.079*"3d" + 0.045*"printer" 
'0.013*"trump" + 0.010*"year" + 0.010*"us" 
0.085*"bigdata" + 0.047*"data" + 0.034*"ai"


RoboticsEU
[(5, 0.71964782964474272), (3, 0.078212371175674852), (10, 0.061950759536093108)]
'0.086*"robot" + 0.026*"manufactur" + 0.014*"engin" 
'0.017*"love" + 0.016*"day" + 0.014*"today"
'0.013*"trump" + 0.010*"year" + 0.010*"us" 

karpathy
[(11, 0.64604950410257989), (15, 0.24672485912868494), (3, 0.05996141157383509)]
'0.013*"time" + 0.010*"thing" + 0.008*"learn" 
'0.044*"learn" + 0.021*"deep" + 0.020*"machin"
'0.017*"love" + 0.016*"day" + 0.014*"today"



polar3d
[(2, 0.62244045071421772), (16, 0.24404842301495563), (3, 0.12745196053857411)]
'0.087*"3dprint" + 0.079*"3d" + 0.045*"printer"
'0.031*"stem" + 0.030*"code" + 0.024*"learn" 
'0.017*"love" + 0.016*"day" + 0.014*"today"


thearduinoguy
[(7, 0.75203259942685219), (3, 0.15457573273012132), (16, 0.03163785260115036)]
'0.051*"arduino" + 0.019*"robot" + 0.017*"project"
'0.017*"love" + 0.016*"day" + 0.014*"today" 
'0.031*"stem" + 0.030*"code" + 0.024*"learn"